Symbolic Regression 

In [ ]:
# Install dependencies (run once)
!pip install pysr numpy pandas scikit-learn matplotlib seaborn plotly

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Setup complete")

In [ ]:
from pysr import PySRRegressor

print(f"PySR loaded successfully")

In [ ]:
# Generate synthetic data for kinetic energy
np.random.seed(42)
n_samples = 200

# Random mass and velocity values
m = np.random.uniform(0.1, 10, n_samples)  # mass (kg)
v = np.random.uniform(0.1, 20, n_samples)  # velocity (m/s)

# True kinetic energy with small noise
E_true = 0.5 * m * v**2
noise = np.random.normal(0, 0.01 * E_true.std(), n_samples)
E = E_true + noise

# Create feature matrix
X = np.column_stack([m, v])
y = E

print(f"Data shape: {X.shape}")
print(f"Features: mass (m), velocity (v)")
print(f"Target: kinetic energy (E)")

In [ ]:
# Visualize the data
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

scatter1 = axes[0].scatter(m, E, c=v, cmap='viridis', alpha=0.7)
axes[0].set_xlabel('Mass (kg)')
axes[0].set_ylabel('Kinetic Energy (J)')
axes[0].set_title('Energy vs Mass (colored by velocity)')
plt.colorbar(scatter1, ax=axes[0], label='Velocity (m/s)')

scatter2 = axes[1].scatter(v, E, c=m, cmap='plasma', alpha=0.7)
axes[1].set_xlabel('Velocity (m/s)')
axes[1].set_ylabel('Kinetic Energy (J)')
axes[1].set_title('Energy vs Velocity (colored by mass)')
plt.colorbar(scatter2, ax=axes[1], label='Mass (kg)')

plt.tight_layout()
plt.show()

In [ ]:
# Configure symbolic regression
model = PySRRegressor(
    niterations=40,  # Number of iterations (increase for better results)
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["square", "sqrt", "abs"],
    populations=15,
    population_size=33,
    maxsize=20,  # Max complexity of equations
    parsimony=0.0032,  # Penalty for complexity (Occam's razor)
    random_state=42,
    deterministic=True,
    procs=0,  # Use 0 for deterministic mode
    multithreading=False,
    temp_equation_file=True,  # Don't save intermediate files
)

print("Model configured. Ready to search for equations...")

In [ ]:
# Run symbolic regression 
print("Searching for equations... (this may take a few minutes)")
model.fit(X, y, variable_names=["m", "v"])

In [ ]:
# View discovered equations (Pareto frontier)
print("\n" + "="*60)
print("DISCOVERED EQUATIONS (Pareto Frontier)")
print("="*60)
print("\nEquations ordered by complexity (simpler → more complex):")
print(model)

In [ ]:
# Get the best equation
best_eq = model.sympy()
print(f"\nBest equation found: E = {best_eq}")
print(f"\nTrue equation: E = 0.5 * m * v²")

In [ ]:
# Evaluate model performance
y_pred = model.predict(X)
r2 = r2_score(y, y_pred)
rmse = np.sqrt(mean_squared_error(y, y_pred))

print(f"R² Score: {r2:.6f}")
print(f"RMSE: {rmse:.6f}")

# Parity plot
plt.figure(figsize=(6, 6))
plt.scatter(y, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2, label='Perfect fit')
plt.xlabel('True Energy (J)')
plt.ylabel('Predicted Energy (J)')
plt.title(f'Symbolic Regression: Predicted vs True\nR² = {r2:.4f}')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
def plot_pareto_frontier(model):
    """Plot the Pareto frontier of discovered equations."""
    equations = model.equations_
    
    if equations is None or len(equations) == 0:
        print("No equations found yet. Run model.fit() first.")
        return
    
    fig, ax = plt.subplots(figsize=(10, 6))
    
    complexities = equations['complexity'].values
    losses = equations['loss'].values
    
    # Plot all equations
    scatter = ax.scatter(complexities, losses, s=100, c=range(len(complexities)), 
                         cmap='viridis', edgecolors='black', linewidth=1)
    
    # Connect Pareto frontier
    ax.plot(complexities, losses, 'k--', alpha=0.3)
    
    # Annotate equations
    for i, (c, l) in enumerate(zip(complexities, losses)):
        eq_str = str(equations.iloc[i]['equation'])[:30]  # Truncate long equations
        ax.annotate(f"{i}: {eq_str}", (c, l), textcoords="offset points", 
                   xytext=(5, 5), fontsize=8, alpha=0.8)
    
    ax.set_xlabel('Complexity', fontsize=12)
    ax.set_ylabel('Loss (MSE)', fontsize=12)
    ax.set_title('Pareto Frontier: Accuracy vs. Complexity', fontsize=14)
    ax.set_yscale('log')
    
    plt.tight_layout()
    plt.show()
    
    return equations

eq_df = plot_pareto_frontier(model)

In [ ]:
# Display equations table
if eq_df is not None:
    display_cols = ['complexity', 'loss', 'equation']
    print("\nAll discovered equations:")
    print(eq_df[display_cols].to_string())

In [ ]:
# Simulated materials data based on known physics relationships
np.random.seed(123)
n_materials = 300

# Features (simplified material descriptors)
electronegativity_diff = np.random.uniform(0.1, 2.5, n_materials)  # χ_A - χ_B
avg_ionic_radius = np.random.uniform(0.5, 2.0, n_materials)  # Å
bond_length = np.random.uniform(1.5, 3.5, n_materials)  # Å
coordination_num = np.random.choice([4, 6, 8, 12], n_materials).astype(float)

# Simulated band gap based on a "hidden" formula:
# Eg 1.35 * (Δχ)^1.5 + 0.8 * (r_avg / d) - 0.1 * CN + noise
# This represents simplified physics: electronegativity difference drives gap,
# ionic character (r/d ratio) contributes, coordination reduces gap

Eg_true = (
    1.35 * electronegativity_diff**1.5 + 
    0.8 * (avg_ionic_radius / bond_length) - 
    0.1 * coordination_num
)
Eg_true = np.clip(Eg_true, 0, None)  # Band gap can't be negative
noise = np.random.normal(0, 0.15, n_materials)
Eg = Eg_true + noise
Eg = np.clip(Eg, 0, None)

# Create DataFrame
materials_df = pd.DataFrame({
    'electronegativity_diff': electronegativity_diff,
    'ionic_radius': avg_ionic_radius,
    'bond_length': bond_length,
    'coordination': coordination_num,
    'band_gap': Eg
})

print("Materials Dataset:")
print(materials_df.describe().round(3))

In [ ]:
# Visualize relationships
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].scatter(materials_df['electronegativity_diff'], materials_df['band_gap'], 
                   alpha=0.5, c=materials_df['coordination'], cmap='viridis')
axes[0, 0].set_xlabel('Electronegativity Difference')
axes[0, 0].set_ylabel('Band Gap (eV)')
axes[0, 0].set_title('Band Gap vs Electronegativity Diff')

axes[0, 1].scatter(materials_df['ionic_radius'], materials_df['band_gap'], 
                   alpha=0.5, c=materials_df['bond_length'], cmap='plasma')
axes[0, 1].set_xlabel('Ionic Radius (Å)')
axes[0, 1].set_ylabel('Band Gap (eV)')
axes[0, 1].set_title('Band Gap vs Ionic Radius')

axes[1, 0].scatter(materials_df['bond_length'], materials_df['band_gap'], 
                   alpha=0.5, c=materials_df['electronegativity_diff'], cmap='coolwarm')
axes[1, 0].set_xlabel('Bond Length (Å)')
axes[1, 0].set_ylabel('Band Gap (eV)')
axes[1, 0].set_title('Band Gap vs Bond Length')

# Box plot for coordination number
materials_df.boxplot(column='band_gap', by='coordination', ax=axes[1, 1])
axes[1, 1].set_xlabel('Coordination Number')
axes[1, 1].set_ylabel('Band Gap (eV)')
axes[1, 1].set_title('Band Gap by Coordination Number')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.show()

In [ ]:
# Prepare data for symbolic regression
feature_names = ['chi', 'r', 'd', 'CN']  # Short names for equations
X_mat = materials_df[['electronegativity_diff', 'ionic_radius', 'bond_length', 'coordination']].values
y_mat = materials_df['band_gap'].values

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_mat, y_mat, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Configure model for materials discovery
materials_model = PySRRegressor(
    niterations=50,
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["square", "sqrt", "cube", "abs"],
    populations=20,
    population_size=40,
    maxsize=25,
    parsimony=0.003,
    random_state=42,
    deterministic=True,
    procs=0,
    multithreading=False,
    temp_equation_file=True,
    extra_sympy_mappings={"cube": lambda x: x**3},
)

print("Materials SR model configured.")

In [ ]:
# Run symbolic regression on materials data
print("Searching for band gap formula... (this may take a few minutes)")
materials_model.fit(X_train, y_train, variable_names=feature_names)

In [ ]:
# View discovered equations
print("\n" + "="*60)
print("DISCOVERED BAND GAP EQUATIONS")
print("="*60)
print("\nVariable legend:")
print("  chi = electronegativity difference")
print("  r   = ionic radius")
print("  d   = bond length")
print("  CN  = coordination number")
print("\n")
print(materials_model)

In [ ]:
# Best equation
best_materials_eq = materials_model.sympy()
print(f"\nDiscovered equation: Eg = {best_materials_eq}")
print(f"\nTrue hidden formula: Eg = 1.35*χ^1.5 + 0.8*(r/d) - 0.1*CN")

In [ ]:
# Evaluate on test set
y_train_pred = materials_model.predict(X_train)
y_test_pred = materials_model.predict(X_test)

train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Training R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")
print(f"Test RMSE: {test_rmse:.4f} eV")

# Parity plots
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].scatter(y_train, y_train_pred, alpha=0.5, label='Train')
axes[0].plot([0, y_train.max()], [0, y_train.max()], 'r--', lw=2)
axes[0].set_xlabel('True Band Gap (eV)')
axes[0].set_ylabel('Predicted Band Gap (eV)')
axes[0].set_title(f'Training Set (R² = {train_r2:.4f})')
axes[0].legend()

axes[1].scatter(y_test, y_test_pred, alpha=0.5, label='Test', color='orange')
axes[1].plot([0, y_test.max()], [0, y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('True Band Gap (eV)')
axes[1].set_ylabel('Predicted Band Gap (eV)')
axes[1].set_title(f'Test Set (R² = {test_r2:.4f})')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Pareto frontier for materials equations
print("\nPareto Frontier for Materials Equations:")
plot_pareto_frontier(materials_model)

In [ ]:
# Install matminer if needed (run once)
!pip install matminer pymatgen

In [ ]:
# Load real bandgap dataset from Matbench
from matminer.datasets import load_dataset

# Load experimental band gap dataset
print("Loading experimental band gap dataset...")
df_bandgap = load_dataset('matbench_expt_gap')

print(f"\nDataset shape: {df_bandgap.shape}")
print(f"Columns: {df_bandgap.columns.tolist()}")
print(f"\nBand gap statistics:")
print(df_bandgap['gap expt'].describe())

# Preview some compositions
print(f"\nSample compositions:")
print(df_bandgap.head(10))

In [ ]:
# Convert composition strings to pymatgen Composition objects
from matminer.featurizers.conversions import StrToComposition

print("Converting compositions...")
df_bandgap = StrToComposition().featurize_dataframe(df_bandgap, 'composition')
print("Done! Added 'composition_obj' column")

- Electronegativity: drives ionicity and band gap
- Atomic radius: affects bonding and structure
- Valence electrons: determines electronic structure
- Ionization energy: related to electron binding

In [ ]:
# Generate composition-based features using Matminer
from matminer.featurizers.composition import ElementProperty

# Use a curated set of features (not all 150+ Magpie features)
# This keeps equations interpretable
ep = ElementProperty.from_preset('magpie')

print("Generating features from compositions... (this may take a minute)")
df_features = ep.featurize_dataframe(df_bandgap, 'composition_obj', ignore_errors=True)

# Drop rows with missing features
df_features = df_features.dropna()
print(f"\nDataset after featurization: {df_features.shape[0]} samples")

# Show available features
feature_cols = [c for c in df_features.columns if 'MagpieData' in c]
print(f"Generated {len(feature_cols)} features")

In [ ]:
# Select physically meaningful features for interpretable equations
# Using fewer, more meaningful features leads to better interpretability

selected_features = [
    'MagpieData mean Electronegativity',      # χ_mean
    'MagpieData range Electronegativity',     # Δχ (electronegativity difference)
    'MagpieData mean CovalentRadius',         # r_mean
    'MagpieData range CovalentRadius',        # Δr
    'MagpieData mean NValence',               # valence electrons
    'MagpieData mean Polarizability',         # polarizability
    'MagpieData mean FirstIonizationEnergy',  # ionization energy
]

# Shorter names for equations
short_names = ['χ_mean', 'Δχ', 'r_mean', 'Δr', 'N_val', 'α', 'IE']

# Check which features are available
available_features = [f for f in selected_features if f in df_features.columns]
available_short = [short_names[i] for i, f in enumerate(selected_features) if f in df_features.columns]

print("Selected features for symbolic regression:")
for full, short in zip(available_features, available_short):
    print(f"  {short:8s} = {full}")
    
# Create feature matrix
X_real = df_features[available_features].values
y_real = df_features['gap expt'].values

print(f"\nFeature matrix shape: {X_real.shape}")
print(f"Target (band gap) shape: {y_real.shape}")

In [ ]:
# Visualize feature-target relationships
fig, axes = plt.subplots(2, 3, figsize=(14, 9))
axes = axes.flatten()

for i, (feat, name) in enumerate(zip(available_features[:6], available_short[:6])):
    ax = axes[i]
    ax.scatter(df_features[feat], y_real, alpha=0.3, s=10)
    ax.set_xlabel(name, fontsize=11)
    ax.set_ylabel('Band Gap (eV)')
    ax.set_title(f'Band Gap vs {name}')
    
    # Add correlation
    corr = np.corrcoef(df_features[feat].values, y_real)[0, 1]
    ax.annotate(f'r = {corr:.3f}', xy=(0.05, 0.95), xycoords='axes fraction',
                fontsize=10, va='top')

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("- Electronegativity difference (Δχ) correlates with band gap")
print("- This aligns withknown  physics: larger Δχ → more ionic → larger gap")

In [ ]:
# Prepare data for symbolic regression
from sklearn.preprocessing import StandardScaler

# Train/test split
X_train_real, X_test_real, y_train_real, y_test_real = train_test_split(
    X_real, y_real, test_size=0.2, random_state=42
)

# Normalize features (can help convergence)
# This kinda changes equation coefficients but not structure
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_real)
X_test_scaled = scaler.transform(X_test_real)

print(f"Training samples: {len(X_train_real)}")
print(f"Test samples: {len(X_test_real)}")
print(f"\nUsing raw (unscaled) features for interpretable coefficients")

In [ ]:
# Configure symbolic regression for real bandgap data
bandgap_model = PySRRegressor(
    niterations=60,  # More iterations for real data
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["square", "sqrt", "abs", "exp"],
    populations=20,
    population_size=50,
    maxsize=25,
    parsimony=0.002,  # Slightly lower to allow more complex equations
    random_state=42,
    deterministic=True,
    procs=0,
    multithreading=False,
    temp_equation_file=True,
    # Constraints to avoid nonsensical equations
    nested_constraints={
        "square": {"square": 0, "sqrt": 1},
        "sqrt": {"square": 1, "sqrt": 0},
        "exp": {"exp": 0},
    },
)

print("Bandgap SR model configured for real data.")

In [ ]:
# Run symbolic regression on bandgap data

print("="*60)
print("DISCOVERING BANDGAP EQUATIONS FROM REAL DATA")
print("="*60)
print(f"\nSearching over {len(X_train_real)} materials...")
print("This may take 5-10 minutes. Go grab a coffee!\n")

bandgap_model.fit(X_train_real, y_train_real, variable_names=available_short)

In [ ]:
# View discovered equations
print("\n" + "="*60)
print("DISCOVERED BANDGAP EQUATIONS (Real Data)")
print("="*60)
print("\nVariable legend:")
print("  χ_mean = mean electronegativity")
print("  Δχ     = electronegativity difference (range)")
print("  r_mean = mean covalent radius")
print("  Δr     = covalent radius difference")
print("  N_val  = mean valence electrons")
print("  α      = mean polarizability")
print("  IE     = mean first ionization energy")
print("\n")
print(bandgap_model)

In [ ]:
# Best equation for bandgap
best_bandgap_eq = bandgap_model.sympy()
print(f"Best discovered equation:")
print(f"  Eg = {best_bandgap_eq}")

# Get LaTeX version for papers
from sympy import latex
print(f"\nLaTeX format (for papers):")
print(f"  E_g = {latex(best_bandgap_eq)}")

In [ ]:
# Evaluate on test set
y_train_pred_real = bandgap_model.predict(X_train_real)
y_test_pred_real = bandgap_model.predict(X_test_real)

train_r2_real = r2_score(y_train_real, y_train_pred_real)
test_r2_real = r2_score(y_test_real, y_test_pred_real)
test_rmse_real = np.sqrt(mean_squared_error(y_test_real, y_test_pred_real))
test_mae_real = np.mean(np.abs(y_test_real - y_test_pred_real))

print("Model Performance")
print(f"  Training R²: {train_r2_real:.4f}")
print(f"  Test R²:     {test_r2_real:.4f}")
print(f"  Test RMSE:   {test_rmse_real:.3f} eV")
print(f"  Test MAE:    {test_mae_real:.3f} eV")

In [ ]:
# Parity plots for real data
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Training set
axes[0].scatter(y_train_real, y_train_pred_real, alpha=0.3, s=15)
axes[0].plot([0, 10], [0, 10], 'r--', lw=2, label='Perfect fit')
axes[0].set_xlabel('Experimental Band Gap (eV)', fontsize=11)
axes[0].set_ylabel('Predicted Band Gap (eV)', fontsize=11)
axes[0].set_title(f'Training Set (R² = {train_r2_real:.4f})')
axes[0].set_xlim(0, 10)
axes[0].set_ylim(0, 10)
axes[0].legend()

# Test set
axes[1].scatter(y_test_real, y_test_pred_real, alpha=0.3, s=15, color='orange')
axes[1].plot([0, 10], [0, 10], 'r--', lw=2, label='Perfect fit')
axes[1].set_xlabel('Experimental Band Gap (eV)', fontsize=11)
axes[1].set_ylabel('Predicted Band Gap (eV)', fontsize=11)
axes[1].set_title(f'Test Set (R² = {test_r2_real:.4f}, RMSE = {test_rmse_real:.2f} eV)')
axes[1].set_xlim(0, 10)
axes[1].set_ylim(0, 10)
axes[1].legend()

plt.suptitle('Symbolic Regression on Real Experimental Band Gaps', fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Pareto frontier for real bandgap data
print("Pareto Frontier for Real Bandgap Equations:")
plot_pareto_frontier(bandgap_model)

In [ ]:
# Error analysis: where does the model succeed/fail?
residuals = y_test_real - y_test_pred_real

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Residual distribution
axes[0].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='r', linestyle='--', lw=2)
axes[0].set_xlabel('Residual (eV)')
axes[0].set_ylabel('Count')
axes[0].set_title('Residual Distribution')
axes[0].annotate(f'Mean: {np.mean(residuals):.3f} eV\nStd: {np.std(residuals):.3f} eV',
                xy=(0.95, 0.95), xycoords='axes fraction', ha='right', va='top',
                fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat'))

# Residual vs predicted
axes[1].scatter(y_test_pred_real, residuals, alpha=0.3, s=15)
axes[1].axhline(0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Predicted Band Gap (eV)')
axes[1].set_ylabel('Residual (eV)')
axes[1].set_title('Residuals vs Predictions')

plt.tight_layout()
plt.show()

In [ ]:
# Simpler equations are easier to understand and generalize; complex ones fit better but may overfit.
# Compare equations at different complexity levels
equations_df = bandgap_model.equations_

print("="*70)
print("EQUATION COMPARISON: Simple → Complex")
print("="*70)

for idx, row in equations_df.iterrows():
    eq = row['equation']
    complexity = row['complexity']
    loss = row['loss']
    
    # Calculate test set performance for each equation
    y_pred_eq = bandgap_model.predict(X_test_real, index=idx)
    r2_eq = r2_score(y_test_real, y_pred_eq)
    rmse_eq = np.sqrt(mean_squared_error(y_test_real, y_pred_eq))
    
    print(f"\n[Complexity {complexity}] Test R² = {r2_eq:.4f}, RMSE = {rmse_eq:.3f} eV")
    print(f"  Equation: Eg = {eq}")

In [ ]:
# query from materials project EXAMPLE 

def query_materials_project(api_key, num_materials=1000):
    """
    Query band gaps directly from Materials Project.
    Get API key at: https://materialsproject.org/api
    """
    from mp_api.client import MPRester
    
    with MPRester(api_key) as mpr:
        # Query materials with band gap data
        docs = mpr.summary.search(
            band_gap=(0.1, 10),  # Filter for semiconductors/insulators
            fields=["material_id", "formula_pretty", "band_gap", 
                    "formation_energy_per_atom", "energy_above_hull"],
            num_chunks=1,
            chunk_size=num_materials
        )
    
    # Convert to DataFrame
    df = pd.DataFrame([{
        'material_id': d.material_id,
        'composition': d.formula_pretty,
        'band_gap': d.band_gap,
        'formation_energy': d.formation_energy_per_atom,
        'e_above_hull': d.energy_above_hull
    } for d in docs])
    
    return df

# Example usage:
# df_mp = query_materials_project("API_KEY_HERE!", num_materials=2000)
# df_mp_features = load_and_featurize_csv_from_df(df_mp)

print("Function defined: query_materials_project()")